In [1]:
# --- Notebook 1: global imports, style, and reproducibility helpers ---
# (safe to run multiple times)

import os
import sys
import math
import time
import platform
from pathlib import Path
from typing import Union, Tuple, Callable, Optional, Sequence
import matplotlib as mpl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  # optional; used for nicer defaults in local Jupyter
from scipy.optimize import root_scalar
from tqdm.notebook import tqdm

# -----------------------------
# Reproducibility & paths
# -----------------------------
SEED = 42

def set_global_seed(seed: int = 42):
    """Set seeds for numpy; return a Generator for downstream use."""
    np.random.seed(seed)
    return np.random.default_rng(seed)

rng = set_global_seed(SEED)

# Output directories
PROJ_ROOT = Path.cwd()
OUT_DIR = PROJ_ROOT / "outputs"
FIG_DIR = OUT_DIR / "figs"
TAB_DIR = OUT_DIR / "tables"
for d in (OUT_DIR, FIG_DIR, TAB_DIR):
    d.mkdir(parents=True, exist_ok=True)

# -----------------------------
# Matplotlib / pandas display
# -----------------------------
plt.style.use("seaborn-v0_8-ticks")  # works with modern Matplotlib without seaborn installed
plt.rcParams.update({
    "figure.figsize": (9, 6),
    "figure.dpi": 120,
    "axes.titlesize": 16,
    "axes.labelsize": 13,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "legend.fontsize": 11,
    "legend.title_fontsize": 12,
    "savefig.bbox": "tight",
})

# Optional seaborn theme (keeps Matplotlib in control; comment out if undesired)
sns.set_theme(context="notebook", style="ticks")

# pandas display (nice for sanity checks; doesn't affect computations)
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 160)

# -----------------------------
# Small utilities: timing & env dump
# -----------------------------
class Timer:
    def __enter__(self):
        self.t0 = time.perf_counter()
        return self
    def __exit__(self, *exc):
        self.elapsed = time.perf_counter() - self.t0


def env_summary() -> dict:
    import multiprocessing as mp
    try:
        import psutil  # optional
        mem = getattr(psutil, "virtual_memory", lambda: None)()
        mem_gb = None if mem is None else round(mem.total/1e9, 2)
    except Exception:
        mem_gb = None
    return {
        "python": sys.version.split(" (" )[0],
        "platform": platform.platform(),
        "cpu_count": os.cpu_count(),
        "ram_gb": mem_gb,
        "numpy": np.__version__,
        "pandas": pd.__version__,
        "matplotlib": plt.matplotlib.__version__,
        "scipy": root_scalar.__module__.split(".")[0] + ":" + __import__("scipy").__version__,
        "seaborn": sns.__version__,
        "tqdm": tqdm.__module__.split(".")[0] + ":" + __import__("tqdm").__version__,
        "logical_cores": mp.cpu_count(),
    }

ENV_INFO = env_summary()
print("[ENV]", ENV_INFO)

# Convenience RNG helper for modules that prefer Generator inputs
random = rng  # alias for downstream code

SEED = 42
rng = np.random.default_rng(SEED)

from pathlib import Path
OUT_DIR = Path("outputs"); FIG_DIR = OUT_DIR/"figs"; TAB_DIR = OUT_DIR/"tables"
for d in (OUT_DIR, FIG_DIR, TAB_DIR): d.mkdir(parents=True, exist_ok=True)

plt.rcParams["figure.dpi"] = 120
plt.rcParams["savefig.bbox"] = "tight"


# === Simulation Parameters & Constants (Notebook 1) ===
from dataclasses import dataclass
from pathlib import Path
import numpy as np

# Small-scale exact analysis
NUM_INSTANCES_SMALL: int = 10000   # exact enumeration budget (small n)
N_VALUES_SMALL = [2, 3, 4]        # players for exact small-scale

# Medium/large-scale sampling-based experiments
NUM_INSTANCES_MEDIUM: int = 1000  # budget per (n, p, d, rule) combo
N_VALUES_LARGE = [5, 8, 10, 15, 20, 30]

# Key hyper-parameters
P_VALUES = [0.5, 0.9]             # participation probabilities
D_VALUES = [2, 3, 4]              # polynomial degree of c_e(x) = a x^d
BRD_MAX_ITER = 1_000              # BRD iteration cap per restart
BRD_RESTARTS = 10                 # multi-starts per instance
EPS = 1e-9                        # float tolerance (equality / stopping)

# Sanity checks (fail fast if typo)
assert all(isinstance(n, int) and n >= 2 for n in N_VALUES_SMALL + N_VALUES_LARGE)
assert all(0 < p <= 1 for p in P_VALUES)
assert all(d >= 2 for d in D_VALUES)
assert BRD_MAX_ITER > 0 and BRD_RESTARTS >= 1 and EPS > 0

# Reproducible seeds per high-level sweep (optional but handy)
SEED_GRID = {
    "small": 20250101,
    "medium": 20250102,
    "large": 20250103,
}

# === Paths for artifacts ===
RESULTS_DIR = OUT_DIR / "results"  # OUT_DIR defined in the setup cell
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

FIG_DIR.mkdir(parents=True, exist_ok=True)
TAB_DIR.mkdir(parents=True, exist_ok=True)

# === Plotting style helpers ===
COLORS = {
    "SV": "#0072B2",    # Shapley Value - Blue
    "PS": "#D55E00",    # Proportional Sharing - Burnt Orange
    2: "#0072B2",
    3: "#D55E00",
    4: "#009E73",
    10: "#CC79A7",
    "limit": "black",
}

# Golden ratio constants
PHI = (1 + np.sqrt(5)) / 2
PHI_SQUARED = float(PHI ** 2)

# Compact config dataclass (optional, used by generators)
@dataclass(frozen=True)
class RunConfig:
    n: int
    p: float
    d: int
    rule: str  # "SV" or "PS"
    seed: int

    def tag(self) -> str:
        return f"n{self.n}_p{self.p:g}_d{self.d}_{self.rule}_s{self.seed}"

print("[PARAMS] small n:", N_VALUES_SMALL, "large n:", N_VALUES_LARGE)
print("[PARAMS] P_VALUES:", P_VALUES, "D_VALUES:", D_VALUES)
print("[PATHS] RESULTS_DIR:", RESULTS_DIR)
print("[CONST] PHI^2:", PHI_SQUARED)

# === Theory bounds & root helpers ===
# Robust, vector-friendly implementations for:
#   - C_func_hom(p, n)
#   - poa_hom_quadratic(p, n)
#   - z_star_sv / poa_2player_sv
#   - z_star_ps / poa_2player_ps
# Notes:
# * Accept both scalars and numpy arrays for p.
# * Root-finding uses an expanding bracket to avoid ValueError from brentq.

from typing import Union, Tuple, Callable
import numpy as np
from scipy.optimize import root_scalar

ArrayLike = Union[float, int, np.ndarray]

# ---- small numeric helpers ----
_EPS = EPS  # from params cell


def _to_array(x: ArrayLike) -> np.ndarray:
    """Ensure numpy float array without modifying scalars' shape when possible."""
    return np.asarray(x, dtype=float)


# ---- Homogeneous quadratic bound C(p,n) and PoA ----

def C_func_hom(p: ArrayLike, n: int) -> ArrayLike:
    """C(p,n) from Theorem 1 (Homogeneous, Quadratic).
    Accepts scalar or array p. Returns same shape as p.
    For n < 2, returns NaN.
    """
    p_arr = _to_array(p)
    if n < 2:
        return np.full(p_arr.shape, np.nan)
    m1 = n // 2
    m2 = n - m1
    # (m1, m2) >= 1 since n>=2
    numerator = p_arr**2
    denom = 4.0 * (p_arr + (1.0 - p_arr) / m1) * (p_arr + (1.0 - p_arr) / m2)
    out = np.empty_like(p_arr, dtype=float)
    np.divide(numerator, denom, out=out, where=denom != 0)
    return out if isinstance(p, np.ndarray) else float(out.item())


def poa_hom_quadratic(p: ArrayLike, n: int) -> ArrayLike:
    """PoA upper bound from Theorem 1 (Homogeneous, Quadratic).
    Vectorized in p; handles p=0 -> PoA=1.
    """
    p_arr = _to_array(p)
    C_val = C_func_hom(p_arr, n)
    poa = (np.sqrt(C_val) + np.sqrt(C_val + 1.0))**2
    # handle p==0 → 1 (even though typical runs use p>0)
    if np.isscalar(p):
        return 1.0 if abs(float(p)) <= _EPS else float(poa)
    poa = np.where(np.abs(p_arr) <= _EPS, 1.0, poa)
    return poa


# ---- Root finding with expanding bracket ----

def _bracket_then_brentq(f: Callable[[float], float],
                         lo: float = 1e-12,
                         hi0: float = 1.0,
                         max_hi: float = 1e8,
                         max_doubles: int = 60) -> Tuple[bool, float]:
    """Find a bracketing interval [lo, hi] with sign change, expanding hi exponentially;
    then run brentq. Returns (ok, root_or_nan).
    Assumes f(lo) > 0 and f(hi) eventually < 0 for our use cases.
    """
    try:
        flo = f(lo)
    except Exception:
        return False, np.nan
    hi = hi0
    for _ in range(max_doubles):
        try:
            fhi = f(hi)
        except Exception:
            return False, np.nan
        if np.isnan(flo) or np.isnan(fhi):
            return False, np.nan
        if flo * fhi < 0:  # bracket found
            try:
                sol = root_scalar(f, bracket=[lo, hi], method="brentq")
                return (sol.converged, sol.root if sol.converged else np.nan)
            except Exception:
                return False, np.nan
        hi *= 2.0
        if hi > max_hi:
            break
    return False, np.nan


# ---- 2-player SV ----

def z_star_sv(p: float, d: int) -> float:
    """Root z*>0 for 2-player SV (Theorem 7):
        (1 - p/2) + (p/2)*(1+z)^d - (1 + p/2)*z^d = 0
       For p≈0 → 1, return z*=1.
    """
    p = float(p)
    if p <= _EPS:
        return 1.0
    def f(z: float) -> float:
        return (1.0 - p/2.0) + (p/2.0)*(1.0 + z)**d - (1.0 + p/2.0)*z**d
    ok, root = _bracket_then_brentq(f, lo=1e-12, hi0=1.0)
    return float(root) if ok else np.nan


def poa_2player_sv(p: float, d: int) -> float:
    z = z_star_sv(p, d)
    return float(z**d) if np.isfinite(z) else np.nan


# ---- 2-player PS ----

def z_star_ps(p: float, d: int) -> float:
    """Root z*>0 for 2-player PS (Theorem 9):
        (1 - p) + p*(1+z)^{d-1} - z^d = 0
       For d=2, identical to SV; for p≈0 → 1.
    """
    p = float(p)
    if p <= _EPS:
        return 1.0
    if d == 2:
        return z_star_sv(p, d)
    def f(z: float) -> float:
        return (1.0 - p) + p*(1.0 + z)**(d - 1) - z**d
    ok, root = _bracket_then_brentq(f, lo=1e-12, hi0=1.0)
    return float(root) if ok else np.nan


def poa_2player_ps(p: float, d: int) -> float:
    z = z_star_ps(p, d)
    return float(z**d) if np.isfinite(z) else np.nan


# ---- Quick sanity checks ----
if __name__ == "__main__":
    for n in (2, 3, 10):
        for p in (0.0, 0.5, 1.0):
            C = C_func_hom(p, n)
            PoA = poa_hom_quadratic(p, n)
            print(f"n={n}, p={p}: C={C:.4f}, PoA={PoA:.4f}")
    for d in (2, 3, 4):
        for p in (0.1, 0.5, 0.9):
            zsv = z_star_sv(p, d); psv = poa_2player_sv(p, d)
            zps = z_star_ps(p, d); pps = poa_2player_ps(p, d)
            print(f"d={d}, p={p}: z_SV={zsv:.4f}, PoA_SV={psv:.4f} | z_PS={zps:.4f}, PoA_PS={pps:.4f}")

# ================================================================
# Part 1: Instance Generation & Exact Cost Calculation (with Exact SV)
# ================================================================
from __future__ import annotations
import itertools
from typing import Dict, Tuple, List, Any
import math
import numpy as np

# 全局容差（与主 Notebook 保持一致）
try:
    EPS = EPS  # noqa: F821  # 若上文已定义则沿用
except Exception:
    EPS = 1e-9

# ------------------------------
# 随机实例生成（去重 & 可复现）
# ------------------------------

def generate_random_instance(
    n: int,
    num_strategies_per_player: int,
    num_resources: int,
    rng: np.random.Generator,
) -> Dict[str, Any]:
    """
    生成一个随机博弈实例。
    - 为每个玩家生成若干条不重复路径（随机子集），避免重复策略。
    - 权重 w_i ~ U[0.5, 2.0]
    - 边系数 a_e ~ U[0.5, 1.5]
    """
    instance = {
        'n': n,
        'num_resources': num_resources,
        'num_strategies_per_player': num_strategies_per_player,
        'weights': rng.uniform(0.5, 10.0, size=n),
        'a_coeffs': rng.uniform(0.5, 3, size=num_resources),
        'strategies': []  # List[List[Tuple[int, ...]]]
    }
    resources = list(range(num_resources))
    for _ in range(n):
        player_strategies: List[Tuple[int, ...]] = []
        seen = set()
        # 尽量生成去重的路径；若资源很少且策略数很多，可能出现生成困难，此处简单重试若干次
        trials = 0
        while len(player_strategies) < num_strategies_per_player and trials < 10_000:
            trials += 1
            k = int(rng.integers(1, num_resources + 1))
            path = tuple(sorted(rng.choice(resources, size=k, replace=False)))
            if path not in seen:
                seen.add(path)
                player_strategies.append(path)
        if len(player_strategies) < num_strategies_per_player:
            # 退而求其次：若极端情况下仍不足，重复最后一个（保持维度一致）
            while len(player_strategies) < num_strategies_per_player:
                player_strategies.append(player_strategies[-1])
        instance['strategies'].append(player_strategies)
    return instance


# ----------------------------------------------
# 精确成本计算：枚举 2^n 类型 + 精确 SV（d>2 为排列法）
# ----------------------------------------------

def calculate_exact_costs(
    instance: Dict[str, Any],
    profile: Tuple[int, ...],
    p: float,
    d: int,
    rule: str,
) -> Dict[str, Any]:
    """
    计算无条件期望成本（玩家成本向量 + 社会成本）。
    - rule='PS': 比例分摊
    - rule='SV': Shapley 值（d=2 用闭式，d>2 用排列精确计算）
    仅用于小规模 n（例如 ≤4）的穷举分析。
    """
    rule = rule.upper()
    if rule not in {"PS", "SV"}:
        raise ValueError(f"Unknown rule '{rule}'. Use 'PS' or 'SV'.")

    n = instance['n']
    weights: np.ndarray = np.asarray(instance['weights'], dtype=float)
    a_coeffs: np.ndarray = np.asarray(instance['a_coeffs'], dtype=float)
    num_resources = int(instance['num_resources'])

    # 固定策略剖面对应的路径（每个玩家选一条）
    strategies: List[List[Tuple[int, ...]]] = instance['strategies']
    player_paths: Tuple[Tuple[int, ...], ...] = tuple(
        strategies[i][profile[i]] for i in range(n)
    )

    # 预计算：每条边在该 profile 下的潜在使用者集合（提升一点点性能）
    users_by_edge: List[set[int]] = []
    for e in range(num_resources):
        users = {i for i in range(n) if e in player_paths[i]}
        users_by_edge.append(users)

    total_expected_player_costs = np.zeros(n, dtype=float)

    # 穷举 2^n 类型向量
    for mask in range(1 << n):
        # types[j] ∈ {0,1}
        types = [(mask >> j) & 1 for j in range(n)]
        # 概率（同质 p）
        prob = 1.0
        for t in types:
            prob *= (p if t == 1 else (1 - p))
        if prob < 1e-16:
            continue

        active_players = {idx for idx, t in enumerate(types) if t == 1}
        if not active_players:
            continue  # 无活跃玩家则所有边分摊为 0

        for e in range(num_resources):
            if not users_by_edge[e]:
                continue  # 该 profile 下无人选择此边

            active_on_edge = active_players & users_by_edge[e]
            if not active_on_edge:
                continue

            a_e = a_coeffs[e]

            if rule == 'PS':
                total_w = float(np.sum(weights[list(active_on_edge)]))
                if total_w <= 0:
                    continue
                edge_cost = a_e * (total_w ** d)
                for i in active_on_edge:
                    share = edge_cost * (weights[i] / total_w)
                    total_expected_player_costs[i] += prob * share

            else:  # 'SV'
                if d == 2:
                    # 闭式：chi_i = a_e * w_i * W
                    total_w = float(np.sum(weights[list(active_on_edge)]))
                    for i in active_on_edge:
                        share = a_e * weights[i] * total_w
                        total_expected_player_costs[i] += prob * share
                else:
                    # 精确排列（|S|!），仅用于小 n
                    players_on_edge = list(active_on_edge)
                    m = len(players_on_edge)
                    if m == 1:
                        # 只有自己时，SV = c(w_i)
                        i = players_on_edge[0]
                        share = a_e * (weights[i] ** d)
                        total_expected_player_costs[i] += prob * share
                    else:
                        fact_m = math.factorial(m)
                        for i in players_on_edge:
                            total_mc = 0.0
                            w_i = float(weights[i])
                            for perm in itertools.permutations(players_on_edge):
                                # i 的前置集合的总权重
                                idx_i = perm.index(i)
                                if idx_i == 0:
                                    w_pred = 0.0
                                else:
                                    w_pred = float(np.sum([weights[j] for j in perm[:idx_i]]))
                                total_mc += a_e * ((w_pred + w_i) ** d - (w_pred) ** d)
                            share = total_mc / fact_m
                            total_expected_player_costs[i] += prob * share

    social_cost = float(np.sum(total_expected_player_costs))
    return {'player_costs': total_expected_player_costs, 'social_cost': social_cost}


# ======================================================================
# Part 2: 单实例分析（检查 BNE 并返回非平凡 PoA 的 max/avg）
# ======================================================================

from itertools import product

def run_small_scale_analysis_for_instance(
    instance: Dict[str, Any],
    p: float,
    d: int,
) -> Dict[str, float]:
    """
    穷举 profile，计算 PS 与 SV（精确）下的 BNE，并输出非平凡 PoA 的 max / avg。
    注意：此函数假设同质概率 p；若需要异质 p_i，请扩展 calculate_exact_costs。
    """
    n = int(instance['n'])
    num_strats = int(instance['num_strategies_per_player'])
    all_profiles = list(product(range(num_strats), repeat=n))

    # 预计算所有 profile 成本（避免重复调用）
    profile_costs_ps: Dict[Tuple[int, ...], Dict[str, Any]] = {}
    profile_costs_sv: Dict[Tuple[int, ...], Dict[str, Any]] = {}
    for prof in all_profiles:
        profile_costs_ps[prof] = calculate_exact_costs(instance, prof, p, d, 'PS')
        profile_costs_sv[prof] = calculate_exact_costs(instance, prof, p, d, 'SV')

    opt_ps = min(res['social_cost'] for res in profile_costs_ps.values()) if profile_costs_ps else math.inf
    opt_sv = min(res['social_cost'] for res in profile_costs_sv.values()) if profile_costs_sv else math.inf

    bne_poas_ps: List[float] = []
    bne_poas_sv: List[float] = []

    for prof in all_profiles:
        # PS: 检查是否为 BNE（允许等价收益，加入 EPS 容忍）
        is_bne_ps = True
        costs_here_ps = profile_costs_ps[prof]['player_costs']
        for i in range(n):
            s_curr = prof[i]
            best = costs_here_ps[i]
            for s_alt in range(num_strats):
                if s_alt == s_curr:
                    continue
                prof_alt = list(prof)
                prof_alt[i] = s_alt
                alt = profile_costs_ps[tuple(prof_alt)]['player_costs'][i]
                if alt + EPS < best:
                    is_bne_ps = False
                    break
            if not is_bne_ps:
                break
        if is_bne_ps and opt_ps > EPS:
            bne_poas_ps.append(profile_costs_ps[prof]['social_cost'] / opt_ps)

        # SV: 同理
        is_bne_sv = True
        costs_here_sv = profile_costs_sv[prof]['player_costs']
        for i in range(n):
            s_curr = prof[i]
            best = costs_here_sv[i]
            for s_alt in range(num_strats):
                if s_alt == s_curr:
                    continue
                prof_alt = list(prof)
                prof_alt[i] = s_alt
                alt = profile_costs_sv[tuple(prof_alt)]['player_costs'][i]
                if alt + EPS < best:
                    is_bne_sv = False
                    break
            if not is_bne_sv:
                break
        if is_bne_sv and opt_sv > EPS:
            bne_poas_sv.append(profile_costs_sv[prof]['social_cost'] / opt_sv)

    nontrivial_ps = [x for x in bne_poas_ps if x > 1 + EPS]
    nontrivial_sv = [x for x in bne_poas_sv if x > 1 + EPS]

    return {
        'max_poa_ps': (max(nontrivial_ps) if nontrivial_ps else float('nan')),
        'avg_poa_ps': (float(np.mean(nontrivial_ps)) if nontrivial_ps else float('nan')),
        'max_poa_sv': (max(nontrivial_sv) if nontrivial_sv else float('nan')),
        'avg_poa_sv': (float(np.mean(nontrivial_sv)) if nontrivial_sv else float('nan')),
    }


# ======================================================================
# Part 3: 小规模主实验（缓存文件名改为 exactSV 以示区分）
# ======================================================================

from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm

RESULTS_DIR = Path('results')
RESULTS_DIR.mkdir(exist_ok=True)

def run_small_scale_experiment(
    n: int,
    p: float,
    d: int,
    num_instances_target: int,
    force_rerun: bool = False,
    seed: int = 42,
) -> pd.DataFrame:
    """
    运行小规模穷举实验：累计到指定数量的“有效”实例（至少一个非平凡 PoA 的 BNE）后停止。
    结果缓存为 CSV。
    """
    fname = RESULTS_DIR / f"small_scale_n{n}_p{int(round(p*100))}_d{d}_exactSV.csv"
    if fname.exists() and not force_rerun:
        print(f"Loading cached results from {fname} ...")
        return pd.read_csv(fname)

    rng = np.random.default_rng(seed)
    results: List[Dict[str, float]] = []
    pbar = tqdm(total=num_instances_target, desc=f"Small-scale n={n}, p={p}, d={d}")
    total_runs = 0

    while len(results) < num_instances_target:
        total_runs += 1
        inst = generate_random_instance(
            n=n, num_strategies_per_player=2, num_resources=3, rng=rng
        )
        res = run_small_scale_analysis_for_instance(inst, p, d)
        is_valid = (not np.isnan(res['max_poa_ps'])) or (not np.isnan(res['max_poa_sv']))
        if is_valid:
            results.append({'n': n, 'p': p, 'd': d, **res})
            pbar.update(1)
    pbar.close()

    print(f"Finished. Generated {total_runs} instances to obtain {num_instances_target} valid ones.")
    df = pd.DataFrame(results)
    df.to_csv(fname, index=False)
    print(f"Saved to {fname}")
    return df

# Seed-safe orchestrator for small-scale grid
# - Provides independent or paired instance sampling across (n, p, d)
# - Drop-in replacement for your previous run_small_scale_grid_and_tightness

import numpy as np
import time, json
from pathlib import Path

# assumes the following are already defined in your notebook:
#   RESULTS_DIR, build_tightness_table, capture_env_info, run_small_scale_experiment
#   N_VALUES_SMALL, P_VALUES, D_VALUES

RESULTS_DIR = OUT_DIR / "results"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

def bootstrap_stat(values: np.ndarray,
                   stat_fn: Callable[[np.ndarray], float],
                   B: int = 2000,
                   alpha: float = 0.05,
                   seed: int = 1234) -> dict:
    """Nonparametric bootstrap for a scalar statistic.
    Returns: dict(mean=..., ci_low=..., ci_high=..., samples=[...]).
    Notes:
      - NaN values are removed. If <2 valid samples: returns point estimate, CI=NaN.
      - For 'max' statistic we just apply stat_fn to each resample (size=n with replacement).
    """
    vals = np.asarray(values, dtype=float)
    vals = vals[~np.isnan(vals)]
    n = len(vals)
    if n == 0:
        return {"mean": np.nan, "ci_low": np.nan, "ci_high": np.nan, "samples": []}

    # If only one valid sample, CI is undefined under resampling-with-replacement
    if n == 1:
        est = float(stat_fn(vals))
        return {"mean": est, "ci_low": np.nan, "ci_high": np.nan, "samples": [est]}

    rng = np.random.default_rng(seed)
    boots = np.empty(B, dtype=float)
    for b in range(B):
        idx = rng.integers(0, n, size=n)
        sample = vals[idx]
        boots[b] = float(stat_fn(sample))
    lo = float(np.quantile(boots, alpha/2))
    hi = float(np.quantile(boots, 1 - alpha/2))
    return {"mean": float(np.mean(boots)), "ci_low": lo, "ci_high": hi, "samples": boots}

# --- Helper: theoretical bounds dispatcher ---
def theoretical_bound(rule: str, n: int, p: float, d: int) -> Optional[float]:
    """Return the theoretical PoA upper bound for (rule, n, p, d) when available.
    Rules:
      - d == 2: use homogeneous quadratic bound for both SV and PS (they coincide).
      - n == 2 and d >= 2: use the closed forms from the 2-player theorems.
      - Otherwise: return np.nan (not available in this notebook's exact-analysis scope).
    """
    rule = str(rule).upper()
    if d == 2:
        # Homogeneous quadratic bound (Theorem 1). Both rules same.
        return float(poa_hom_quadratic(p, n))
    if n == 2:
        if rule == 'SV':
            return float(poa_2player_sv(p, d))
        if rule == 'PS':
            return float(poa_2player_ps(p, d))
    return np.nan


def capture_env_info() -> dict:
    return {
        "python": sys.version.split()[0],
        "numpy": np.__version__,
        "pandas": pd.__version__,
        "scipy": __import__('scipy').__version__,
        "matplotlib": mpl.__version__,
        "seaborn": sns.__version__,
        "platform": platform.platform(),
        "processor": platform.processor(),
        "machine": platform.machine(),
    }
    # if psutil is not None:
    #     try:
    #         vm = psutil.virtual_memory()
    #         info.update({
    #             "ram_total_gb": round(vm.total / (1024**3), 2),
    #             "ram_available_gb": round(vm.available / (1024**3), 2),
    #         })
    #     except Exception:
    #         pass
    return info

# --- Build tightness table from per-instance summaries ---
def build_tightness_table(df_list: Sequence[pd.DataFrame],
                          alpha: float = 0.05,
                          B: int = 2000,
                          seed: int = 2025) -> pd.DataFrame:
    """Aggregate multiple cached per-instance CSVs into a tightness table.
    Each input df has rows: {n, p, d, max_poa_ps, avg_poa_ps, max_poa_sv, avg_poa_sv} per instance.
    We aggregate by (graph_class, n, p, d, rule) and report:
      - observed_max, CI
      - observed_avg, CI
      - theoretical_upper_bound
      - gap_to_bound (for max & avg)
      - counts and NaN ratios
    """
    df_all = pd.concat(df_list, ignore_index=True)
    # For this notebook, small-scale generator is path-based random; mark graph class accordingly
    df_all['graph_class'] = 'random-paths'

    rows = []
    keys = sorted(set(zip(df_all['graph_class'], df_all['n'], df_all['p'], df_all['d'])))
    for (gcls, n, p, d) in keys:
        sub = df_all[(df_all['graph_class']==gcls)&(df_all['n']==n)&(df_all['p']==p)&(df_all['d']==d)]
        for rule, col_max, col_avg in [('PS','max_poa_ps','avg_poa_ps'), ('SV','max_poa_sv','avg_poa_sv')]:
            vals_max = sub[col_max].to_numpy(dtype=float)
            vals_avg = sub[col_avg].to_numpy(dtype=float)
            # Observed statistics
            obs_max = np.nanmax(vals_max) if np.any(~np.isnan(vals_max)) else np.nan
            obs_avg = np.nanmean(vals_avg) if np.any(~np.isnan(vals_avg)) else np.nan
            # Bootstrap CIs (deterministic via seed)
            ci_max = bootstrap_stat(vals_max, np.nanmax, B=B, alpha=alpha, seed=seed)
            ci_avg = bootstrap_stat(vals_avg, np.nanmean, B=B, alpha=alpha, seed=seed+1)
            # Theory bound
            bound = theoretical_bound(rule, int(n), float(p), int(d))
            # Gaps
            gap_max = (bound - obs_max) if (not np.isnan(bound) and not np.isnan(obs_max)) else np.nan
            gap_avg = (bound - obs_avg) if (not np.isnan(bound) and not np.isnan(obs_avg)) else np.nan
            rows.append({
                'graph': gcls,
                'n': int(n), 'p': float(p), 'd': int(d), 'Rule': rule,
                'observed_max_poa': obs_max,
                'max_ci_low': ci_max['ci_low'], 'max_ci_high': ci_max['ci_high'],
                'observed_avg_poa': obs_avg,
                'avg_ci_low': ci_avg['ci_low'], 'avg_ci_high': ci_avg['ci_high'],
                'theory_bound': bound,
                'gap_to_bound_max': gap_max,
                'gap_to_bound_avg': gap_avg,
                'count': int(len(sub)),
                'nan_frac_max': float(np.mean(np.isnan(vals_max))) if len(vals_max)>0 else np.nan,
                'nan_frac_avg': float(np.mean(np.isnan(vals_avg))) if len(vals_avg)>0 else np.nan,
            })
    tight = pd.DataFrame(rows)
    # canonical ordering for downstream table formatting
    tight = tight.sort_values(["graph","p","n","d","Rule"]).reset_index(drop=True)
    return tight

def _derive_cfg_seed(base_seed: int, n: int, p: float, d: int, independent: bool = True) -> int:
    """Derive a deterministic 32-bit seed for a config.
    - independent=True: unique seed for each (n,p,d)
    - independent=False: same seed for all configs sharing n (paired comparison across p,d)
    Using SeedSequence keeps it portable and reproducible.
    """
    # quantize p to avoid float representation wobble in the key
    p_key = int(round(float(p) * 1_000_000)) if independent else 0
    d_key = int(d) if independent else 0
    ss = np.random.SeedSequence([int(base_seed), int(n), int(p_key), int(d_key)])
    return int(ss.generate_state(1, dtype=np.uint32)[0])



def run_small_scale_grid_and_tightness(
    n_values=N_VALUES_SMALL,
    p_values=P_VALUES,
    d_values=D_VALUES,
    instances_each: int = 200,
    force_rerun: bool = False,
    base_seed: int = 42,
    independent_instances: bool = True,
    alpha: float = 0.05,
    B: int = 2000,
):
    """Run the grid of small-scale experiments with seed safety.

    Parameters
    ----------
    independent_instances : bool
        - True  -> each (n,p,d) gets an independent RNG seed (statistically independent).
        - False -> all configs that share the same n reuse the *same* instance pool
                   (paired comparisons across p,d; useful to reduce variance when comparing settings).
    base_seed : int
        Master seed from which per-config seeds are deterministically derived.
    """
    t0 = time.time()
    dfs = []

    # Collect per-config seeds for auditability
    seed_book = []

    for n in n_values:
        for p in p_values:
            for d in d_values:
                cfg_seed = _derive_cfg_seed(base_seed, int(n), float(p), int(d), independent=independent_instances)
                seed_book.append({"n": int(n), "p": float(p), "d": int(d), "seed": int(cfg_seed)})

                df = run_small_scale_experiment(
                    n=int(n), p=float(p), d=int(d),
                    num_instances_target=int(instances_each),
                    force_rerun=force_rerun,
                    seed=int(cfg_seed),
                )
                dfs.append(df)

    # Tightness table
    tight = build_tightness_table(dfs, alpha=alpha, B=B, seed=base_seed + 7)

    # Save artifacts
    env = capture_env_info()
    tight_path = RESULTS_DIR / 'tightness_small_scale.csv'
    env_path = RESULTS_DIR / 'env_small_scale.json'
    seeds_path = RESULTS_DIR / 'seeds_small_scale.json'
    tight.to_csv(tight_path, index=False)
    with open(env_path, 'w') as f:
        json.dump(env, f, indent=2)
    with open(seeds_path, 'w') as f:
        json.dump(seed_book, f, indent=2)

    print(f"Saved tightness table -> {tight_path}")
    print(f"Saved environment specs -> {env_path}")
    print(f"Saved per-config seeds -> {seeds_path}")
    print(f"Total wall-clock: {time.time() - t0:.1f}s")

    return tight

# Usage examples (pick one):
# 1) Independent sampling across the whole grid (recommended for unbiased distributional summaries)
# tight = run_small_scale_grid_and_tightness(independent_instances=True)
# 2) Paired sampling across p,d within each n (same instance pool per n; good for fair A/B comparisons)
# tight = run_small_scale_grid_and_tightness(independent_instances=False)


[ENV] {'python': '3.12.7 | packaged by Anaconda, Inc. |', 'platform': 'macOS-15.6.1-arm64-arm-64bit', 'cpu_count': 8, 'ram_gb': 17.18, 'numpy': '1.26.4', 'pandas': '2.2.2', 'matplotlib': '3.9.2', 'scipy': 'scipy:1.13.1', 'seaborn': '0.13.2', 'tqdm': 'tqdm:4.66.5', 'logical_cores': 8}
[PARAMS] small n: [2, 3, 4] large n: [5, 8, 10, 15, 20, 30]
[PARAMS] P_VALUES: [0.5, 0.9] D_VALUES: [2, 3, 4]
[PATHS] RESULTS_DIR: outputs/results
[CONST] PHI^2: 2.618033988749895
n=2, p=0.0: C=0.0000, PoA=1.0000
n=2, p=0.5: C=0.0625, PoA=1.6404
n=2, p=1.0: C=0.2500, PoA=2.6180
n=3, p=0.0: C=0.0000, PoA=1.0000
n=3, p=0.5: C=0.0833, PoA=1.7676
n=3, p=1.0: C=0.2500, PoA=2.6180
n=10, p=0.0: C=0.0000, PoA=1.0000
n=10, p=0.5: C=0.1736, PoA=2.2500
n=10, p=1.0: C=0.2500, PoA=2.6180
d=2, p=0.1: z_SV=1.0512, PoA_SV=1.1051 | z_PS=1.0512, PoA_PS=1.1051
d=2, p=0.5: z_SV=1.2808, PoA_SV=1.6404 | z_PS=1.2808, PoA_PS=1.6404
d=2, p=0.9: z_SV=1.5466, PoA_SV=2.3919 | z_PS=1.5466, PoA_PS=2.3919
d=3, p=0.1: z_SV=1.1049, PoA_SV

In [6]:
import matplotlib as mpl # <<< FIX: Added the required import
# Example usage in a separate cell (keep heavy runs explicit):
tight = run_small_scale_grid_and_tightness(instances_each=10000, force_rerun=True, independent_instances=False)
display(tight.head())


Small-scale n=2, p=0.5, d=2:   0%|          | 0/10000 [00:00<?, ?it/s]

Finished. Generated 300508 instances to obtain 10000 valid ones.
Saved to outputs/results/small_scale_n2_p50_d2_exactSV.csv


Small-scale n=2, p=0.5, d=3:   0%|          | 0/10000 [00:00<?, ?it/s]

Finished. Generated 304016 instances to obtain 10000 valid ones.
Saved to outputs/results/small_scale_n2_p50_d3_exactSV.csv


Small-scale n=2, p=0.5, d=4:   0%|          | 0/10000 [00:00<?, ?it/s]

...

Small-scale n=4, p=0.9, d=3:   0%|          | 0/10000 [00:00<?, ?it/s]

Finished. Generated 139440 instances to obtain 10000 valid ones.
Saved to outputs/results/small_scale_n4_p90_d3_exactSV.csv


Small-scale n=4, p=0.9, d=4:   0%|          | 0/10000 [00:00<?, ?it/s]

Finished. Generated 132781 instances to obtain 10000 valid ones.
Saved to outputs/results/small_scale_n4_p90_d4_exactSV.csv
Saved tightness table -> outputs/results/tightness_small_scale.csv
Saved environment specs -> outputs/results/env_small_scale.json
Saved per-config seeds -> outputs/results/seeds_small_scale.json
Total wall-clock: 5810.0s


,graph,n,p,d,Rule,observed_max_poa,max_ci_low,max_ci_high,observed_avg_poa,avg_ci_low,avg_ci_high,theory_bound,gap_to_bound_max,gap_to_bound_avg,count,nan_frac_max,nan_frac_avg
0,random-paths,2,0.5,2,PS,1.489209,1.445080,1.489209,1.068891,1.067704,1.070057,1.640388,0.151179,0.571498,10000,0.0000,0.0000
1,random-paths,2,0.5,2,SV,1.489209,1.445080,1.489209,1.068891,1.067704,1.070057,1.640388,0.151179,0.571498,10000,0.0000,0.0000
2,random-paths,2,0.5,3,PS,2.414634,2.382862,2.414634,1.181701,1.178014,1.185743,3.766177,1.351543,2.584476,10000,0.0542,0.0542
3,random-paths,2,0.5,3,SV,2.414634,2.353706,2.414634,1.177918,1.174267,1.181675,4.136631,1.721997,2.958714,10000,0.0398,0.0398
4,random-paths,2,0.5,4,PS,4.422150,4.206269,4.422150,1.371484,1.361445,1.381472,12.401529,7.979379,11.030044,10000,0.0939,0.0939


In [8]:
import numpy as np
import pandas as pd
from typing import List

EPS = 1e-9

def format_latex_table_from_tight_v2(
    tight_df: pd.DataFrame,
    filename: str,
    rules: List[str] = ["PS", "SV"],
    digits: int = 3,
    highlight_best: bool = True,
) -> str:
    """
    Build a LaTeX table with, per rule: Ave PoA (CI), Max PoA, Upper Bound, Gap.
    Uses your existing columns in `tight`:
      - 'p','n','d','rule','observed_avg_poa','avg_ci_low','avg_ci_high',
        'observed_max_poa','theory_bound','gap_to_bound_max'
    If 'gap_to_bound_max' is absent, computes as theory_bound - observed_max_poa (floored at 0).
    """
    req_cols = [
        'p','n','d','Rule',
        'observed_avg_poa','avg_ci_low','avg_ci_high',
        'observed_max_poa',
        'theory_bound'
    ]
    for c in req_cols:
        if c not in tight_df.columns:
            raise ValueError(f"Column '{c}' missing from tight_df")

    df = tight_df.copy()
    # Normalize rule labels to upper for matching
    # df['Rule'] = df['rule'].astype(str).str.upper()

    # Compute gap if missing
    if 'gap_to_bound_max' not in df.columns:
        df['gap_to_bound_max'] = df['theory_bound'] - df['observed_max_poa']
        # Ensure non-negative gap (if desired)
        df['gap_to_bound_max'] = df['gap_to_bound_max'].apply(lambda x: x if pd.isna(x) else max(0.0, x))

    df_sorted = df.sort_values(["p","n","d","Rule"]).reset_index(drop=True)

    # --- Header ---
    lines = []
    lines.append('% Auto-generated LaTeX table: Ave PoA (CI), Max PoA, Bound, Gap')
    lines.append('\\begin{table}[ht!]')
    lines.append('\\centering')
    lines.append('\\caption{Average and maximum observed PoA with 95\\% CI, theoretical bounds, and gaps.}')
    lines.append('\\label{tab:tightness_table_ave_max_bound_gap}')
    lines.append('\\footnotesize')
    lines.append('\\renewcommand{\\arraystretch}{1.15}')

    num_rules = len(rules)
    col_spec = 'ccc' + 'cccc' * num_rules

    header1 = ['\\multirow{2}{*}{$p$}', '\\multirow{2}{*}{$n$}', '\\multirow{2}{*}{$d$}']
    cmidrules = []
    header2 = ['','','']

    for i, rule_name in enumerate(rules):
        header1.append(f'\\multicolumn{{4}}{{c}}{{{rule_name}}}')
        start_col = 4 + i*4
        end_col = 7 + i*4
        cmidrules.append(f'\\cmidrule(lr){{{start_col}-{end_col}}}')
        header2.extend(['Ave PoA (CI)', 'Max PoA', 'Upper Bound', 'Gap'])

    lines.append(f'\\begin{{tabular}}{{{col_spec}}} \\toprule')
    lines.append(' & '.join(header1) + ' \\')
    lines.append(' '.join(cmidrules))
    lines.append(' & '.join(header2) + ' \\')
    lines.append('\\midrule')

    # --- Body ---
    unique_params = list(dict.fromkeys(zip(df_sorted['p'], df_sorted['n'], df_sorted['d'])))

    for p_val, n_val, d_val in unique_params:
        # Best (min) max PoA among rules for highlighting
        row_block = df_sorted[(df_sorted['p']==p_val)&(df_sorted['n']==n_val)&(df_sorted['d']==d_val)]
        min_max = np.inf
        if highlight_best and not row_block.empty and row_block['observed_max_poa'].notna().any():
            min_max = row_block['observed_max_poa'].min()

        cells = [f'{p_val:.2f}', f'{int(n_val)}', f'{int(d_val)}']

        for rule in rules:
            sub = row_block[row_block['Rule']==rule]
            if sub.empty:
                cells.extend(['$*$','$*$','$*$','$*$'])
                continue
            r = sub.iloc[0]
            # Ave PoA (CI)
            ave = r.get('observed_avg_poa', np.nan)
            lo = r.get('avg_ci_low', np.nan)
            hi = r.get('avg_ci_high', np.nan)
            if pd.isna(ave) or pd.isna(lo) or pd.isna(hi):
                ave_ci_str = '$*$'
            else:
                ave_ci_str = f"{ave:.{digits}f} [{lo:.{digits}f}, {hi:.{digits}f}]"

            # Max PoA with highlight (bold if equal to row min)
            maxv = r.get('observed_max_poa', np.nan)
            if pd.isna(maxv):
                max_str = '$*$'
            else:
                max_str = f"{maxv:.{digits}f}"
                if highlight_best and np.isfinite(min_max) and abs(maxv - min_max) < EPS:
                    max_str = f"\\textbf{{{max_str}}}"

            # Bound & Gap
            bound = r.get('theory_bound', np.nan)
            gap = r.get('gap_to_bound_max', np.nan)
            bound_str = '$*$' if pd.isna(bound) else f"{bound:.{digits}f}"
            gap_str = '$*$' if pd.isna(gap) else f"{gap:.{digits}f}"

            cells.extend([ave_ci_str, max_str, bound_str, gap_str])

        lines.append(' & '.join(cells) + ' \\')

    lines.append('\\bottomrule')
    lines.append('\\end{tabular}')
    lines.append('\\end{table}')

    latex = '\n'.join(lines)
    with open(filename, 'w') as f:
        f.write(latex)
    print(f"LaTeX table saved to '{filename}'")
    return latex

# --- Usage example (uncomment when running in your notebook) ---
latex_code = format_latex_table_from_tight_v2(
    tight,
    filename=str(RESULTS_DIR / 'tightness_table.tex'),
    rules=['PS','SV'],
    digits=3,
    highlight_best=False,
)
print(latex_code)



LaTeX table saved to 'outputs/results/tightness_table.tex'
% Auto-generated LaTeX table: Ave PoA (CI), Max PoA, Bound, Gap
\begin{table}[ht!]
\centering
\caption{Average and maximum observed PoA with 95\% CI, theoretical bounds, and gaps.}
\label{tab:tightness_table_ave_max_bound_gap}
\footnotesize
\renewcommand{\arraystretch}{1.15}
\begin{tabular}{ccccccccccc} \toprule
\multirow{2}{*}{$p$} & \multirow{2}{*}{$n$} & \multirow{2}{*}{$d$} & \multicolumn{4}{c}{PS} & \multicolumn{4}{c}{SV} \
\cmidrule(lr){4-7} \cmidrule(lr){8-11}
 &  &  & Ave PoA (CI) & Max PoA & Upper Bound & Gap & Ave PoA (CI) & Max PoA & Upper Bound & Gap \
\midrule
0.50 & 2 & 2 & 1.069 [1.068, 1.070] & 1.489 & 1.640 & 0.151 & 1.069 [1.068, 1.070] & 1.489 & 1.640 & 0.151 \
0.50 & 2 & 3 & 1.182 [1.178, 1.186] & 2.415 & 3.766 & 1.352 & 1.178 [1.174, 1.182] & 2.415 & 4.137 & 1.722 \
0.50 & 2 & 4 & 1.371 [1.361, 1.381] & 4.422 & 12.402 & 7.979 & 1.350 [1.341, 1.360] & 4.422 & 18.376 & 13.954 \
0.50 & 3 & 2 & 1.051 [1.050, 1.

In [9]:
import platform
import os
import sys
import numpy
import pandas
import scipy
import matplotlib
import tqdm
import joblib
import numba

# 使用 try-except 来处理可选的 psutil 库
try:
    import psutil
except ImportError:
    psutil = None
try:
    import seaborn
except ImportError:
    seaborn = None

def get_system_specs():
    """收集并返回关键的硬件和软件信息。"""
    specs = {}

    # 1. 硬件信息
    specs['OS'] = platform.platform()
    specs['CPU'] = platform.processor()
    specs['Logical Cores'] = os.cpu_count()
    if psutil:
        specs['Physical Cores'] = psutil.cpu_count(logical=False)
        ram_gb = psutil.virtual_memory().total / (1024**3)
        specs['RAM (GB)'] = f"{ram_gb:.2f}"
    else:
        specs['Physical Cores'] = "N/A (psutil not installed)"
        specs['RAM (GB)'] = "N/A (psutil not installed)"

    # 2. 软件信息
    specs['Python'] = platform.python_version()
    
    packages = {
        "NumPy": numpy, "SciPy": scipy, "pandas": pandas,
        "Matplotlib": matplotlib, "Seaborn": seaborn, "tqdm": tqdm,
        "Joblib": joblib, "Numba": numba
    }
    
    package_versions = []
    for name, pkg in packages.items():
        if pkg:
            package_versions.append(f"{name} v{pkg.__version__}")
    
    specs['Key Packages'] = ", ".join(package_versions)

    return specs

if __name__ == "__main__":
    print("--- System Specifications ---")
    system_info = get_system_specs()
    for key, value in system_info.items():
        print(f"{key:<15}: {value}")
    
    if not psutil:
        print("\n[NOTE] To get Physical Core count and RAM, please install psutil: pip install psutil")

--- System Specifications ---
OS             : macOS-15.6.1-arm64-arm-64bit
CPU            : arm
Logical Cores  : 8
Physical Cores : 8
RAM (GB)       : 16.00
Python         : 3.12.7
Key Packages   : NumPy v1.26.4, SciPy v1.13.1, pandas v2.2.2, Matplotlib v3.9.2, Seaborn v0.13.2, tqdm v4.66.5, Joblib v1.4.2, Numba v0.60.0
